In [ ]:
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier

from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

from tqdm import tqdm
tqdm.pandas()

In [5]:
path="../data/"
train = pd.read_parquet(path+"train.parquet")

train, test = train_test_split(train, test_size=0.2, random_state=1)
eval, test = train_test_split(test, test_size=0.5, random_state=1)

sub = pd.read_csv(path+"sample_submission.csv")

In [6]:
train.head()

,id,prompt,response_a,response_b,winner,model_a,model_b,language
18632,623e2aa50f13aece7fb1c60cf555d8c7d290692f679655...,Phân tích kết quả sản xuất (KQSX) về chất lượn...,Để phân tích kết quả sản xuất (KQSX) về chất l...,Phân tích kết quả sản xuất (KQSX) về chất lượn...,model_a,yi-lightning,internlm2_5-20b-chat,Vietnamese
34859,b87079eef09873032f653314cfddd05e7bf77e4e6d0449...,Write a sample of Old Uyghur,Providing a truly representative sample of Old...,"I apologize, but I cannot generate an authenti...",model_a,gemini-1.5-flash-002,claude-3-5-sonnet-20240620,English
43764,e721d5bfbff8e05e2ffa92e0137a570c1b1b7fa3f7bbf7...,Write a poem in the style of Ada Limon.,**Under the Bruise of the World**\n\nIn the ga...,Here's a poem inspired by the style of Ada Lim...,model_b,reka-core-20240722,grok-2-2024-08-13,English
22541,77350a005d0195fa951b9fb073b018f2f6424a58ee58b5...,"describe the way ""golden slumbers"" by the beat...","Describing the way ""Golden Slumbers"" by The Be...","For a deaf person who has never heard ""Golden ...",model_a,jamba-1.5-large,deepseek-coder-v2-0724,English
37157,c43d63ee99a8b776065f410e4a1e0791a4449230cfd648...,かけっこをしています。あなたは3位の人を抜かし ました。今あなたは何位？,3位です。あなたは3位の人を抜かしたので、今あなたは3位になります。,はい、かけっこの状況について考えてみましょう。\n\n問題文から、以下のことがわかります。\...,model_a,o1-preview,claude-3-opus-20240229,Japanese


In [7]:
test.head()

,id,prompt,response_a,response_b,winner,model_a,model_b,language
7419,271ba404fc25609b8ceb6f37c3375f278c42e2479514ea...,Улучшь сообщение: Добрый вечер! \nЭто Алексей ...,"Добрый вечер!\n\nМеня зовут Алексей Королев, я...","Добрый вечер!\n\nЯ, Алексей Королев, заместите...",model_b,qwen-max-0919,c4ai-aya-expanse-32b,Russian
39112,ce7be103ac8f040accc9b8ec421a6cbf78189c503671df...,i have a notification icon in my webpage. when...,Certainly! To implement a notification system ...,Here's a simple example of how you can achieve...,model_b,qwen2.5-72b-instruct,llama-3.1-405b-instruct-bf16,English
33946,b39725e0a7992eb0c5f34542f960727a4fd306741568fb...,I have a raspberry pi that is booted from USB....,**Hotplugging the SD Card on a Raspberry Pi**\...,"No, you generally **cannot** safely hotplug th...",model_a,llama-3.1-8b-instruct,gemini-1.5-pro-exp-0827,English
35392,bb33f84e8e85da9e41fa271f6210fdffba0400d725a04c...,"Imagine video on YouTube ""Asking people what i...",Here are some imagined YouTube comments relati...,Here are some comments that might appear under...,model_a,gemini-1.5-flash-002,gemma-2-27b-it,English
10254,364c8146a6a236fbdfa34492dba10405a0e448d3062316...,generate a wine description,Please provide some context! To generate a co...,**Wine Description:**\n\n**Name:** Sunset on t...,model_b,gemini-1.5-flash-8b-001,llama-3.1-70b-instruct,English


In [8]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-multilingual-cased")

model.cls = nn.Identity()

Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
def get_text_token_length(text: str, tokenizer) -> int:
    tok_out = tokenizer(text)
    return len(tok_out[0])

In [57]:
train_prompt_length = train['prompt'].progress_apply(get_text_token_length, args=(tokenizer,))
train_response_a_length = train['response_a'].progress_apply(get_text_token_length, args=(tokenizer,))
train_response_b_length = train['response_b'].progress_apply(get_text_token_length, args=(tokenizer,))

100%|██████████| 48439/48439 [00:30<00:00, 1613.45it/s]


In [7]:
print(train_prompt_length.max(), train_prompt_length.min())
plt.hist(train_prompt_length, bins=200);

NameError: name 'train_prompt_length' is not defined

In [10]:
class QADataset(Dataset):
    def __init__(self, data, device):
        self.data = data
        self.device = device
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        encoded = (
            tokenizer(row["prompt"], return_tensors="pt", padding="max_length", truncation=True),
            tokenizer(row["response_a"], return_tensors="pt", padding="max_length", truncation=True),
            tokenizer(row["response_b"], return_tensors="pt", padding="max_length", truncation=True),
        )

        return (
            {key: tensor.squeeze(0).to(self.device) for key, tensor in encoded[0].items()},
            {key: tensor.squeeze(0).to(self.device) for key, tensor in encoded[1].items()},
            {key: tensor.squeeze(0).to(self.device) for key, tensor in encoded[2].items()},
        )
    

In [11]:
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-multilingual-cased")

model.cls = nn.Identity()
model = model.to(device)

batch_size = 32
train_data_loader = DataLoader(QADataset(train, device), batch_size=batch_size, shuffle=True, drop_last=True)
eval_data_loader = DataLoader(QADataset(eval, device), batch_size=batch_size, drop_last=True)
test_data_loader = DataLoader(QADataset(test, device), batch_size=batch_size, drop_last=True)



Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
@torch.no_grad()
def get_dataloader_embeddings(dataloader: DataLoader, model):
    embeddings = {
        "prompt": [], 
        "response_a": [], 
        "response_b": [], 
    }

    model.eval()

    for batch in tqdm(dataloader):
        prompts = batch[0]
        responses_a = batch[1]
        responses_b = batch[2]

        prompts_embs = model(**prompts)
        response_a_embs = model(**responses_a)
        response_b_embs = model(**responses_b)

        # embeddings["prompt"].append(prompts_embs.logits.mean(1).detach().cpu().numpy())
        # embeddings["response_a"].append(response_a_embs.logits.mean(1).detach().cpu().numpy())
        # embeddings["response_b"].append(response_b_embs.logits.mean(1).detach().cpu().numpy())

        embeddings["prompt"].append(prompts_embs.logits[:, 0, :].detach().cpu().numpy())
        embeddings["response_a"].append(response_a_embs.logits[:, 0, :].detach().cpu().numpy())
        embeddings["response_b"].append(response_b_embs.logits[:, 0, :].detach().cpu().numpy())
    
    return embeddings

In [13]:
train_embeddings = get_dataloader_embeddings(train_data_loader, model)
eval_embeddings = get_dataloader_embeddings(eval_data_loader, model)
test_embeddings = get_dataloader_embeddings(test_data_loader, model)

  0%|          | 0/1210 [00:00<?, ?it/s]

100%|██████████| 151/151 [00:52<00:00,  2.86it/s]


In [14]:
# Эмбеддинг документа -- усреднение эмбеддингов токено
train_embeddings = get_dataloader_embeddings(train_data_loader, model)
eval_embeddings = get_dataloader_embeddings(eval_data_loader, model)
test_embeddings = get_dataloader_embeddings(test_data_loader, model)

100%|██████████| 605/605 [00:55<00:00, 10.94it/s]


In [14]:
def aggregate_dataloader_embeddings(embeddings: dict[str, list[np.ndarray]]):
    prompts_embs = np.concatenate(embeddings['prompt'])
    response_a_embs = np.concatenate(embeddings['response_a'])
    response_b_embs = np.concatenate(embeddings['response_b'])

    all_embs = np.concatenate([prompts_embs, response_a_embs, response_b_embs], axis=1)
    return all_embs

train_features = aggregate_dataloader_embeddings(train_embeddings)
eval_features = aggregate_dataloader_embeddings(eval_embeddings)
test_features = aggregate_dataloader_embeddings(test_embeddings)

In [15]:
X_train = train_features
y_train = train['winner'].iloc[:len(X_train)]

X_eval = eval_features
y_eval = eval['winner'].iloc[:len(X_eval)]

X_test = test_features
y_test = test['winner'].iloc[:len(X_test)]


print(len(X_train), len(y_train))
print(len(X_eval), len(y_eval))
print(len(X_test), len(y_test))

38720 38720
4832 4832
4832 4832


In [16]:
cbm = CatBoostClassifier(random_state=42, eval_metric="Accuracy", verbose=100)
cbm.fit(X_train, y_train, eval_set=(X_eval, y_eval))

Learning rate set to 0.078186
0:	learn: 0.5128874	test: 0.5060017	best: 0.5060017 (0)	total: 116ms	remaining: 1m 55s
100:	learn: 0.6655217	test: 0.4935844	best: 0.5132450 (1)	total: 5.11s	remaining: 45.5s
200:	learn: 0.7464360	test: 0.5057947	best: 0.5132450 (1)	total: 9.89s	remaining: 39.3s
300:	learn: 0.8065341	test: 0.4877897	best: 0.5132450 (1)	total: 14.9s	remaining: 34.5s
400:	learn: 0.8525052	test: 0.4952401	best: 0.5132450 (1)	total: 20.7s	remaining: 30.9s
500:	learn: 0.8854597	test: 0.5057947	best: 0.5132450 (1)	total: 26.2s	remaining: 26.1s
600:	learn: 0.9077738	test: 0.5051738	best: 0.5132450 (1)	total: 31.7s	remaining: 21.1s
700:	learn: 0.9251808	test: 0.5097268	best: 0.5132450 (1)	total: 37.2s	remaining: 15.9s
800:	learn: 0.9392045	test: 0.4946192	best: 0.5132450 (1)	total: 42.6s	remaining: 10.6s
900:	learn: 0.9518595	test: 0.5045530	best: 0.5132450 (1)	total: 48.3s	remaining: 5.31s
999:	learn: 0.9614928	test: 0.5060017	best: 0.5132450 (1)	total: 53.7s	remaining: 0us

best